In [1]:
import pandas as pd
import requests
import json
import time
sleep=0.4

In [2]:
LOCAL=True
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [3]:
print(len(df))

57588


## Fix bad `profile_id` for `LOCAL → True`

In [4]:
#author_id='J.S.Useche.1'
def get_profile_id(author_id,sleep=0.4):
    #if True:
    status=False
    l=[]
    pi=None
    #try:
    #if True:
    try:
        #https://inspirehep.net/literature?q=ea+J.D.Ruiz.Alvarez.2+and+ac+1->10
        url=f'https://inspirehep.net/api/literature?q=ea+{author_id}+and+ac+1->10'
        r=requests.get(url)
        time.sleep(sleep)
        if r.status_code==200:
            data=r.json()
            status=True
            #if not `try` fails and `pass``.
        for d in data.get('hits').get('hits'):
            #l=[a.get('record') for a in d.get('metadata').get('authors') if a.get('bai')==author_id]
            try: #failsafe
                l=[a for a in data.get('hits').get('hits')[0].get('metadata').get('authors') if a.get('bai')==author_id]
            except:
                l=[]
        if len(l)==1 and l[0].get('record'):
            pi=l[0].get('record').get('$ref').split('/')[-1]
    except:
        pass
    return pi,status

TEST=True
if TEST:
    assert get_profile_id('J.S.Useche.1')==('2572877',True)
    assert get_profile_id('J.D.Ruiz.Alvarez.2')==('1184534',True)
    assert get_profile_id('S.Das.8')==(None,True)
    assert get_profile_id('M.Sanchez.39')==(None, True)

In [5]:
def get_advisor(i,sleep=0.4):
    adv=[]
    try:
        r=requests.get('https://inspirehep.net/api/authors/{}'.format(i))
        time.sleep(sleep)
        adv=r.json().get('metadata').get('advisors')
        if adv is None:
            adv=[]
    except:
        pass
    return adv

In [6]:
#a=df[0]
#a.get('author_id')
GET_ADVISORS=False
if GET_ADVISORS:
    kk=[a.update({'advisors':get_advisor(a.get('profile_id'))}) for a in df if not a.get('advisors')]

In [ ]:
SAVE=False
status=0
step=1000
i=0
for start in range(0,len(df),step):
    for d in df[start:start+step]:
        i+=1
        print(str(i).zfill(5),status,end='\r')

        if len(set([dd.get('author_id') for dd in df  if dd.get('profile_id')==d.get('profile_id')]))==1:
            continue
        author_id=d.get('author_id')
        check_profile_id=get_profile_id(author_id)
        if check_profile_id[1] and check_profile_id[0]!=d.get('profile_id'):
            d['profile_id']=check_profile_id[0]
            d['advisors']=get_advisor(d.get('profile_id'))
            status=1
        else:
            status=0
    if SAVE:
        f=open('data/inspire_LA.json','w')
        json.dump(df,f)
        f.close()

## Students object

In [5]:
[dd for dd in df if dd.get('author_id')=='M.Sanchez.39']

[{'author_id': 'M.Sanchez.39',
  'country': 'Colombia',
  'email_addresses': None,
  'full_name': 'Sánchez, M.',
  'ids': None,
  'institution': 'Antioquia U.',
  'institution_id': '903906',
  'name': None,
  'papers': [{'recid': 1415059,
    'year': '2015',
    'citation_count': 2,
    'primary_arxiv_category': [],
    'journal_id': '1365972',
    'author_id': 'M.Sanchez.39',
    'instituion_id': '903906'}],
  'positions': None,
  'profile_id': None,
  'advisors': []}]

In [4]:
df=pd.DataFrame(df)

In [5]:
df[df.advisors.apply(len)>0].shape

(11326, 12)

In [6]:
df[ (df['positions'].str[0].str['rank'].apply(lambda r: r is not None)) & 
   (df['positions'].str[0].str['rank'].apply(lambda r: r is not None)) ].shape

(13745, 12)

In [7]:
df[df['positions'].str[0].apply(lambda r:isinstance(r,dict))].shape

(28388, 12)

People which have edited their profile in INSPIREHEP

In [8]:
df[ (df['positions'].str[0].str['rank'].apply(lambda r:isinstance(r,str))) 
    | (df.advisors.apply(len)>0 ) ].shape

(16762, 12)

In [9]:
df.shape

(57588, 12)

In [10]:
#df['profile_id']=df['profile_id'].fillna(0).astype(int).astype(str).apply(lambda p: None if p=='0' else p)

In [11]:
df[df['profile_id']=='991924']

,author_id,country,email_addresses,full_name,ids,institution,institution_id,name,papers,positions,profile_id,advisors
4722,D.Restrepo.1,Colombia,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...",Antioquia U.,903906,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
4762,D.Restrepo.1,Italy,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...","ICTP, Trieste",902861,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
5942,D.Restrepo.1,Spain,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego A.","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...",Valencia U.,903319,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
6064,D.Restrepo.1,Brazil,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...","IIP, Brazil",912344,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
6066,D.Restrepo.1,Brazil,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...",Campinas State U.,902714,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
6072,D.Restrepo.1,Spain,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, D.A.","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...","Valencia U., IFIC",907907,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."


In [12]:
profile_id = '991924'
a = {'ids': [{'value': 'INSPIRE-00133260', 'schema': 'INSPIRE ID'}],
 'name': 'Valle, Jose W.F.',
 'hidden': False,
 'record': {'$ref': 'https://inspirehep.net/api/authors/985058'},
 'degree_type': 'phd',
 'curated_relation': False}
r = {'author_id':'D.Restrepo.1',
   'name':{'value': 'Restrepo Quintero, Diego Alejandro',
   'preferred_name': 'Diego Restrepo'},
   'positions': [{'rank': 'PHD',
                  'hidden': False,
                  'record': {'$ref': 'https://inspirehep.net/api/institutions/907907'},
                  'current': False,
                  'end_date': '2001',
                  'start_date': '1997',
                  'institution': 'Valencia U., IFIC',
                  'curated_relation': True}
                ]
   }
advisor_profile_id = '985058'
d = {}

def get_position(ds, p):
    for k in p.keys():
        if k == 'record':
            try:
                ds['institution_id'] = p[k].get('$ref').split('/')[-1]
            except:
                ds['institution_id'] = None
        else:
            ds[k] = p[k]
    return ds


def get_student(rr, a, profile_id,ranks=['UNDERGRADUATE','MASTER','PHD']):
    "Trigger if advisor exits within `a`"
    r=rr.copy()
    students=[]
    degree_type = a.get('degree_type')
    if not r.get('positions'):
        r['positions']=[]
    #    return students
    for p in r.get('positions'):
        ds = {}
        if p.get('rank') and p.get('rank').lower() == degree_type:
            ds = get_position(ds,p)
        elif len(r.get('advisors'))==1 and p.get('rank') and (p.get('rank')=='PHD' or p.get('rank')=='MASTER'):
            ds = get_position(ds,p) 
        else:
            continue
        if ds:
            if ds.get('rank') not in ranks and degree_type:
                ds['rank'] = degree_type.upper()
            ds['student_id'] = r.get('author_id')
            ds['student_name'] = r.get('name').get('value')
            ds['student_profile_id'] = profile_id
            students.append(ds)
    # if not positons in ranks
    if not students:
        #Advisor declared but not in positions
        ds = {}
        ds['rank'] = "PHD"
        if r.get('author_id'):
            ds['student_id'] = r.get('author_id')
            if r.get('name'):
                ds['student_name'] = r.get('name').get('value')
            ds['student_profile_id'] = profile_id
        students.append(ds)


    return students
#assert get_student(r, a, profile_id).get('institution_id') == '907907'

def get_student_without_advisor(p, r, profile_id, ranks=['UNDERGRADUATE','MASTER','PHD']):
    ds = {}
    ds = get_position(ds, p)
    if r.get('author_id'):
        ds['student_id'] = r.get('author_id')
        if r.get('name'):
            ds['student_name'] = r.get('name').get('value')
        ds['student_profile_id'] = profile_id
    return ds


def get_id(d, df, id_type='profile_id'):
    new_id = d.get('profile_id')
    try:
        lpid = list(set(df[ (df['name'].str['value']==d.get('name')) ][id_type].to_list()))
    except:
        lpid=[]
    if len(lpid) == 1:
        if lpid[0]:
            new_id = lpid[0]
    return new_id


def get_advisor(d, advisor_profile_id, a, df):
    d['profile_id'] = advisor_profile_id

    d['name'] = a.get('name')

    if not d.get('profile_id'):
        d['profile_id'] = get_id(d, df, id_type='profile_id')
    if not d.get('profile_id'):
        d['author_id'] = get_id(d, df, id_type='author_id')

    return d
assert get_advisor(d,advisor_profile_id, a, df).get('profile_id') == '985058'

In [13]:
# 991924  # J Valle advisor
# 1854904 # not advisor → Unknown
# 1045947 # not position
# 1014628 # J Valle advisor
# 1028906 # not advisor → Unknown
profile_id=['991924','1014628','1854904','1045947','1021727','1073921','1066143','989075','983327','1009735'][-1]
r=df[df['profile_id']==profile_id].iloc[0].to_dict()

In [14]:
f=open(f'tests/data/{profile_id}.json','w')
json.dump(r,f)
f.close()

In [15]:
r.get('advisors')

[{'name': 'Gleeson, Austin M.', 'degree_type': 'other'}]

In [16]:
r.get('positions')

[{'rank': 'SENIOR',
  'record': {'$ref': 'https://inspirehep.net/api/institutions/902716'},
  'current': True,
  'institution': 'Caracas, U. Central',
  'curated_relation': True},
 {'rank': 'PHD',
  'record': {'$ref': 'https://inspirehep.net/api/institutions/903264'},
  'end_date': '1987',
  'institution': 'Texas U.',
  'curated_relation': True},
 {'rank': 'UNDERGRADUATE',
  'record': {'$ref': 'https://inspirehep.net/api/institutions/903205'},
  'institution': 'Simon Bolivar U.',
  'curated_relation': True}]

In [17]:
def get_students(r, profile_id, pre_students=[]):
    ranks = ['UNDERGRADUATE', 'MASTER', 'PHD']

    if any([ any([ss.get('student_profile_id') == profile_id for ss in s.get('students')])
       for s in pre_students]):
        print('Already in students')
        #raise Exception('Already in students')
        return pre_students

    for a in r.get('advisors'):
        # Get profile_id or author_id or name of advisor
        try:
            advisor_profile_id = a.get('record').get('$ref').split('/')[-1]
        except:
            advisor_profile_id = ''

        OLD_ADVISOR = False
        for old_a in pre_students:
            # TODO: check for other indetifiers
            if advisor_profile_id and old_a.get('profile_id') == advisor_profile_id:
                old_a['students'].extend(get_student(r, a, profile_id,ranks))
                OLD_ADVISOR = True
                break
        if OLD_ADVISOR:
            continue

        d = {}
        d = get_advisor(d, advisor_profile_id, a, df)

        d['students'] = get_student(r, a, profile_id,ranks)

        pre_students.append(d)



    # Update ranks
    if r.get('advisors'):
        L = [[s.get('rank') for s in a.get('students') if s.get('rank')] for a in pre_students]
        ranks = set(ranks).difference([item for sublist in L for item in sublist])

    if not r.get('positions'):
        #raise Exception('Already in students')
        return pre_students


    for p in r.get('positions'):
        if p.get('rank') in ranks:
            d = {}
            d['students'] = []
            ds = get_student_without_advisor(p, r, profile_id, ranks=ranks)
            d['students'].append(ds)
            d['profile_id'] = None
            d['name'] = 'Unkown'
            pre_students.append(d)

    return pre_students


In [18]:
profile_id='991924'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert len(pre_students)==3
profile_id='1014628'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert pre_students[0].get('students')[0].get('rank')=='PHD'
profile_id='1854904'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert len(pre_students)==3
profile_id='1045947'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert pre_students[0].get('profile_id')=='991924'
profile_id='1073921'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
pre_students = get_students(r, profile_id, [])
assert len(set(['MASTER','PHD']).difference(
    [[dd.get('rank') for dd in d.get('students')] for d in pre_students][0]
 ))==0
profile_id='1066143'
f=open(f'tests/data/{profile_id}.json','r')
r=json.load(f)
f.close()
LIST=[[dd.get('rank') for dd in d.get('students')] for d in get_students(r,profile_id, [])]
assert [item for sublist in LIST for item in sublist] == ['PHD', 'PHD']

In [19]:
dfa=df.drop_duplicates(subset=['profile_id']).reset_index(drop=True)

In [20]:
pre_students = []
for profile_id in dfa['profile_id']:
    r = df[df['profile_id']==profile_id]
    if not r.empty:
        r = r.iloc[0].to_dict()
    else:
        continue
    if profile_id and r:
        pre_students = get_students(r, profile_id,pre_students)

Get Advisor level

In [21]:
def add_level(pre_students,LL,level=0):
    for parent_id in LL:
        #print(parent_id,end='\r')
        try:
            d=[d for d in pre_students if d.get('profile_id')==parent_id][0]
            d['level']=level
        except:
            continue

In [22]:
ps=pd.DataFrame(pre_students)
ps=ps.dropna(subset=['profile_id']).reset_index(drop=True)
# students
L=ps.students.apply(lambda L: [d.get('student_profile_id') for d in L]).to_list()
# Level 0 advisors
LL=list(  set(ps.profile_id.to_list()).difference( set([item for subitem in L for item in subitem]) ) ) 
LL=[x for x in LL if x]
print(len(LL))

level=0
while LL:
    add_level(pre_students,LL,level=level)

    LL=[ [dd.get('student_profile_id') for dd in  d.get('students') if dd.get('rank')=='PHD'] for d in pre_students if d.get('level')==level]
    LL=set([item for sublist in LL for item in sublist])
    LL=[x for x in LL if x]
    print(level,len(LL))
    level+=1
    if level>10:
        break

    

2170
0 2662
1 1235
2 430
3 71
4 8
5 4
6 0


In [23]:
chk_levels=['1020782','1013015','989075','996018','1026011','1066143']
LL=[[d.get('level') for d in pre_students if d.get('profile_id')==chk_levels[i]] for i in range(len(chk_levels))]
assert [item for sublist in LL for item in sublist]==list(range(6))

In [24]:
[d for d in pre_students if d.get('level')==4][0]

{'profile_id': '1020446',
 'name': 'Raya Montaño, Alfredo',
 'students': [{'rank': 'PHD',
   'institution_id': '905835',
   'end_date': '2017',
   'start_date': '2013',
   'institution': 'IFM-UMSNH, Michoacan',
   'curated_relation': True,
   'student_id': 'S.Hernandez.Ortiz.1',
   'student_name': 'Hernández-Ortiz, Saúl',
   'student_profile_id': '1274692'},
  {'rank': 'PHD',
   'institution_id': '905835',
   'end_date': '2016',
   'start_date': '2012',
   'institution': 'IFM-UMSNH, Michoacan',
   'curated_relation': True,
   'student_id': 'Aftab.Ahmad.1',
   'student_name': 'Ahmad, Aftab',
   'student_profile_id': '1373106'},
  {'rank': 'MASTER',
   'institution_id': '909842',
   'end_date': '2005',
   'start_date': '2003',
   'institution': 'Punjab U., CHEP',
   'curated_relation': True,
   'student_id': 'Aftab.Ahmad.1',
   'student_name': 'Ahmad, Aftab',
   'student_profile_id': '1373106'}],
 'level': 4}

In [ ]:
f=open('data/students.json','w')
json.dump(pre_students,f)
f.close()

Example: CINVESTAV-IPN

In [883]:
institution_id='903002'
ins_studs=[d for d in pre_students if any([dd.get('institution_id')==institution_id for dd in d.get('students')])]

In [887]:
cinvestav=[]
for d in ins_studs:
    dd=d.copy()
    students=[]
    for s in dd.get('students'):
        if s.get('institution_id')==institution_id:
            students.append(s)
    dd['students']=students
    cinvestav.append(dd)


In [891]:
LL=[d.get('students') for d in cinvestav]
len([item for sublist in LL for item in sublist])

105

,institution,profile_id
0,"Unlisted, VE",1018731
1,"Unlisted, VE",2023397
2,Catolica del Norte U.,2023398
3,"Unlisted, VE",1864347
4,"EAIFR, Kigali",2087213
...,...,...
57583,Rio de Janeiro Federal U.,2516608
57584,Rio de Janeiro Federal U.,1008053
57585,Rio de Janeiro Federal U.,1030697
57586,Rio de Janeiro Federal U.,None


In [937]:
cv=pd.DataFrame(cinvestav)
LL=cv.apply(lambda d: [{'student_name':dd.get('student_name'),
                     'profile_id':dd.get('student_profile_id'),
                     'degree':dd.get('rank'),
                     'start':dd.get('start_date'),
                     'end':dd.get('end_date'),#'student_position':dd.get('position'),
                     'advisor_name':d.get('name'),
                     'advisor_id':d.get('profile_id')} for dd in d.get('students')],axis=1)
LL=[item for sublist in LL for item in sublist]
cv_df=pd.DataFrame(LL)
cv_df=cv_df.sort_values(by=['advisor_name','end']).reset_index(drop=True)
cv_df[:1]

,student_name,profile_id,degree,start,end,advisor_name,advisor_id
0,"Higuita Borja, Daniel Fernando",1318752,MASTER,2010,2012,"Ayón-Beato, Eloy",1017802


In [939]:
cv_df=cv_df.merge(df[['institution','profile_id']],on='profile_id',how='left').drop_duplicates(
    ['student_name','degree','advisor_name']).rename(columns={'institution':'position'}).reset_index(drop=True)
cv_df    

,student_name,profile_id,degree,start,end,advisor_name,advisor_id,position
0,"Higuita Borja, Daniel Fernando",1318752,MASTER,2010,2012,"Ayón-Beato, Eloy",1017802,"Chile Austral U., Valdivia"
1,"Mendez Zavaleta, Julio Alberto",1380842,MASTER,2010,2012,"Ayón-Beato, Eloy",1017802,"Chile Austral U., Valdivia"
2,"Juárez-Aubry, María Montserrat",1762125,PHD,None,2014,"Ayón-Beato, Eloy",1017802,Arkansas State U.
3,"Mendez Zavaleta, Julio Alberto",1380842,PHD,2012,2016,"Ayón-Beato, Eloy",1017802,"Chile Austral U., Valdivia"
4,"Higuita Borja, Daniel Fernando",1318752,PHD,2013,2018,"Ayón-Beato, Eloy",1017802,"Chile Austral U., Valdivia"
...,...,...,...,...,...,...,...,...
100,"Martinez-Castro, Jesus",1065485,PHD,None,None,Unkown,None,"CIC, IPN"
101,"Amador Ceron, Eduardo Xavier",1069816,PHD,None,None,Unkown,None,"CINVESTAV, IPN"
102,"Amador Ceron, Eduardo Xavier",1069816,UNDERGRADUATE,None,None,Unkown,None,"CINVESTAV, IPN"
103,"Padilla Albores, Luis Enrique",1774079,PHD,2014,2020,"Vazquez, José Alberto",1061596,"Mexico U., ICN"


In [940]:
cv_df[cv_df['advisor_name'].str.contains('Roig')]

,student_name,profile_id,degree,start,end,advisor_name,advisor_id,position
71,"Miranda Hernández, Jesús Alejandro",1827003,MASTER,2016,2018,"Roig Garces, Pablo",1049577,"CINVESTAV, IPN"
72,"Miranda Hernández, Jesús Alejandro",1827003,PHD,2018,2022,"Roig Garces, Pablo",1049577,"CINVESTAV, IPN"


In [943]:
cv_df[['student_name', 'profile_id', 'degree', 'start', 'end', 'position','advisor_name',
       'advisor_id', ]].to_csv('data/cinvestav.csv',index=False)

In [941]:
cv_df[cv_df['advisor_name'].str.contains('Zepeda')]

,student_name,profile_id,degree,start,end,advisor_name,advisor_id,position
104,"Villasenor Cendejas, Luis Manuel",1021497,PHD,1981,1988,"Zepeda, Arnulfo",1013360,"IFM-UMSNH, Michoacan"


## Get all the institutions from INSPIREHEP

In [34]:
# united states, england, germany, france, italy, china, spain, japan, korea
# Missing: Caltech; Princeton U.; Valencia U., IFIC; Oxford U.; Cambridge U., DAMTP; Aachen, Tech. Hochsch.; Humboldt U., Berlin; DESY; CERN; Fermilab; SLAC; Heidelberg U.
# Munich, Max Planck Inst.; Hamburg U.; Mainz U., Inst. Phys.; Frankfurt U.; Munich, Tech. U.; Imperial Coll., London; INFN, Turin; INFN, Rome; SISSA, Trieste; Rome U.; 
# Frascati; ICTP, Trieste; Turin U.
def get_institutions(q):
    page=1
    if q:
        url=f'https://inspirehep.net/api/institutions?q={q}&size=250&page={page}'
    else:
        url=f'https://inspirehep.net/api/institutions?size=250&page={page}' 
    r=requests.get(url)
    time.sleep(sleep)
    i=r.json()['hits']['hits']
    total=r.json().get('hits').get('total')
    if not page or page==1:
        ins=[d.get('metadata') for d in i]
    j=page
    next=r.json().get('links').get('next')
    while next:
        print(j)
        r=requests.get(next)
        time.sleep(sleep)
        try:
            i=r.json()['hits']['hits']
        except:
            i=[]
        ins.extend([d.get('metadata') for d in i])
        try:
            next=r.json().get('links').get('next')
        except:
            next=None
        j+=1
        if j > total//250:
            break
    return ins



In [35]:
kk=[]
for q in ['','u','england','germany','france','italy','spain','korea','china','japan','united states',
    'russia','brazil']:
    print(q)
    ins=get_institutions(q)
    for d in ins:
        if d not in kk:
            kk.append(d)
    print(len(kk))
    if len(kk)>11835:
        break            



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
9500
u
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
10701
england
10722
germany
1
2
10778
france
1
2
10886
italy
1
10987
spain
11019
korea
11034
china
1
2
3
11108
japan
1
2
3
4
11193
united states
1
2
3
4
5
6
7
8
9
10
11
11442
russia
1
11499
brazil
11509


In [40]:
ins=kk.copy()

In [37]:
f=open('data/inspire_institutions.json','r')
old=json.load(f)
f.close()

In [41]:
for d in old:
    if d not in kk:
        kk.append(d)

In [50]:
cn=[d.get('control_number') for d in kk]

In [51]:
duplicates=list(set([x for x in cn if cn.count(x)>1]))
len(duplicates)


430

In [46]:
len(set([d.get('control_number') for d in kk]))

11621

In [58]:
ins=[]
for d in kk:
    if d.get('control_number') not in duplicates:
        ins.append(d)

In [59]:
for cn in duplicates:
    ins.append( [d for d in  kk if d.get('control_number')==cn][-1] )

In [60]:
len(ins)

11621

In [68]:
kk=ins.copy()

In [69]:
f=open('data/inspire_institutions.json','w')
json.dump(kk,f)
f.close()

In [70]:
dfi=pd.DataFrame(kk)
dfi=dfi.sort_values('number_of_papers', ascending=False).reset_index(drop=True)

In [71]:
def not_cn(name,sleep=0.4):
    url=f'https://inspirehep.net/api/literature?q=aff+{name}+and+ac+1->10&size=1'
    r=requests.get(url)
    time.sleep(sleep)
    try:
        total=r.json().get('hits').get('total')
    except:
        total=None
    return total

In [72]:
dfi['number_of_not_cn_papers']=dfi.legacy_ICN.apply(not_cn)

In [73]:
dfi['number_of_not_cn_papers']=dfi.number_of_not_cn_papers.fillna(-1).astype(int)

In [74]:
dfi.sort_values(['number_of_not_cn_papers','number_of_papers'], ascending=False)[['legacy_ICN','number_of_not_cn_papers','number_of_papers']][0:50].reset_index(drop=True)#.to_csv('kk.csv',index=False)

,legacy_ICN,number_of_not_cn_papers,number_of_papers
0,CERN,51762,70745
1,Fermilab,22039,35249
2,SLAC,18386,25303
3,Brookhaven,15711,22932
4,"LBL, Berkeley",15152,24309
5,DESY,12933,19280
6,"KEK, Tsukuba",12494,20330
7,Los Alamos,10149,13939
8,Caltech,8744,15051
9,"Munich, Max Planck Inst.",8013,12765


In [77]:
control_number=917747
dfi[dfi['control_number']==control_number].iloc[0].number_of_not_cn_papers

101

In [94]:
for d in ins:
    d['number_of_not_cn_papers']=int(dfi[dfi['control_number']==d.get('control_number')].iloc[0].number_of_not_cn_papers)

In [98]:
f=open('data/inspire_institutions.json','w')
json.dump(ins,f)
f.close()

In [125]:
kkk=pd.DataFrame(kk)
kkk[kkk.legacy_ICN.str.lower().str.contains('valencia')]

,number_of_papers,addresses,ICN,core,self,urls,$schema,legacy_ICN,control_number,legacy_version,...,external_system_identifiers,extra_words,related_records,public_notes,name_variants,deleted_records,inspire_categories,historical_data,inactive,new_record
21,142,"[{'cities': ['Valencia'], 'latitude': 39.47811...",[U. Valencia],NaN,{'$ref': 'https://inspirehep.net/api/instituti...,[{'value': 'http://www.uv.es/'}],https://inspirehep.net/schemas/records/institu...,U. Valencia (main),917747,20161021230226.0,...,NaN,[Univ.],NaN,NaN,[{'value': 'University of Valencia'}],NaN,NaN,NaN,NaN,NaN
3536,2,"[{'state': 'CA', 'cities': ['Valencia'], 'lati...",NaN,NaN,{'$ref': 'https://inspirehep.net/api/instituti...,NaN,https://inspirehep.net/schemas/records/institu...,"Kelly Johnson, Valencia",906045,20161012160154.0,...,"[{'value': 'INST-42552', 'schema': 'SPIRES'}]","[California, research center]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5170,62,"[{'cities': ['Burjassot'], 'country_code': 'ES...",NaN,NaN,{'$ref': 'https://inspirehep.net/api/instituti...,[{'value': 'http://www.uv.es/~webuv/ingles/ind...,https://inspirehep.net/schemas/records/institu...,"Valencia U., Observ.",908764,20130819034633.0,...,"[{'value': 'INST-74314', 'schema': 'SPIRES'}]","[Univ., University, Observatory]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7401,33,"[{'cities': ['Valencia'], 'country_code': 'ES'...",NaN,NaN,{'$ref': 'https://inspirehep.net/api/instituti...,[{'value': 'http://centros.uv.es/web/departame...,https://inspirehep.net/schemas/records/institu...,"Valencia U., Math. Dept.",909221,20130819034656.0,...,"[{'value': 'INST-79278', 'schema': 'SPIRES'}]","[Univ., University, Mathematics, department, M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8395,4,"[{'cities': ['Valencia'], 'latitude': 39.47678...",NaN,False,{'$ref': 'https://inspirehep.net/api/instituti...,[{'value': 'https://www.ucv.es/'}],https://inspirehep.net/schemas/records/institu...,Catholic U. Valencia,943244,20161023033453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8594,36,"[{'cities': ['Valencia'], 'country_code': 'ES'...",NaN,NaN,{'$ref': 'https://inspirehep.net/api/instituti...,[{'value': 'http://www.uv.es/die'}],https://inspirehep.net/schemas/records/institu...,"Valencia U., Dept. Ing. Electronica",911986,20130819034827.0,...,"[{'value': 'INST-109398', 'schema': 'SPIRES'}]","[Universitat Univ., University, Electronic Eng...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
